<a href="https://colab.research.google.com/github/kekenziii/Pengembangan-Sistem-Peramalan-Tingkat-Hunian-Hotel-Menggunakan-LightGBM/blob/main/Sistem_Pengembangan_Sistem_Peramalan_Tingkat_Hunian_Hotel_Menggunakan_LightGBM_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
print("Streamlit")
!pip install streamlit

print("\nPyngrok")
!pip install pyngrok

print("\nHolidays")
!pip install holidays # Buat data liburan di Indonesia

print("\nDarts")
!pip install darts # Buat model

print("\nOptuna")
!pip install optuna # Parameter tuning

## Library Versions

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import plotly
# import holidays
# import calendar
# from datetime import timedelta
# from darts import __version__ as darts_version
# import optuna
# import warnings
# import streamlit as st
# from pyngrok import ngrok

# # Check versions for libraries
# print("Pandas:", pd.__version__)
# print("NumPy:", np.__version__)
# print("Matplotlib:", plt.matplotlib.__version__)
# print("Plotly:", plotly.__version__)
# print("Holidays:", holidays.__version__)
# print("Darts:", darts_version)
# print("Optuna:", optuna.__version__)
# print("Streamlit:", st.__version__)
# print("Pyngrok:", ngrok.__version__)

# # Built-in Python modules (no version attribute)
# print("Calendar: Built-in Python module")
# print("Datetime (timedelta): Built-in Python module")
# print("Warnings: Built-in Python module")

Pandas: 2.2.2

NumPy: 1.26.4

Matplotlib: 3.10.0

Plotly: 5.24.1

Holidays: 0.63

Darts: 0.32.0

Optuna: 4.1.0

Streamlit: 1.41.1

Pyngrok: 7.2.3


Calendar: Built-in Python module

Datetime (timedelta): Built-in Python module

Warnings: Built-in Python module


# Application Code

In [ ]:
%%writefile app.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import plotly.graph_objs as go
import holidays
import calendar
from datetime import timedelta

import statsmodels.api as sm

from darts import TimeSeries
from darts.models import LightGBMModel # Model
from darts.metrics.metrics import mae, smape, rmse # Evaluation

import optuna # Parameter tuning

import warnings # Biar gak diganggu warning
warnings.filterwarnings('ignore')

# Sistem
import streamlit as st # Streamlit
from pyngrok import ngrok # Deploy sistem web based


# System

if "menu" not in st.session_state:
    st.session_state.menu = "Home"

def set_page(page):
    st.session_state.menu = page
    st.rerun()

menu = st.sidebar.selectbox(
    "Select Page",
    ["Home", "Forecasting", "About Us"],
    index=["Home", "Forecasting", "About Us"].index(st.session_state.menu),
)

if menu != st.session_state.menu:
    set_page(menu)


st.markdown(
    f"""
    <style>
    body, [data-testid="stAppViewContainer"] {{
        background-color: white;
        color: #3a3a3a;
        font-family: 'Times New Roman', Times, serif;
    }}

    [data-testid="stSidebar"] {{
        font-family: 'Times New Roman', Times, serif;
    }}

    [data-testid="stHeader"] {{
        background-color: white;
        height: 0px;
        padding: 0px;
        box-shadow: none;
    }}

    .custom-header {{
        background-color: white;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        display: flex;
        justify-content: flex-end;
        align-items: center;
        width: 100%;
        padding: 20px 20px;
        position: fixed;
        top: 0;
        left: 0;
        z-index: 10;
    }}

    .custom-header img {{
        width: 125px;
        height: 100px;
    }}

    .main-content {{
        padding-top: 50px;
    }}
    </style>

    """,
    unsafe_allow_html=True,

    # <div class="custom-header">
    #     <img src="{image_url}" alt="Company Logo">
    # </div>
    # <div class="main-content">
    # </div>
)

st.markdown(
    """
    <style>
    .stButton > button {
        background-color: #5c2e91;
        color: white;
        border: none;
        padding: 10px 20px;
        font-size: 16px;
        font-weight: bold;
        border-radius: 5px;
        cursor: pointer;
    }
    .stButton > button:hover {
        background-color: white;
        color: #5c2e91;
        border: 2px solid #5c2e91;
    }
    </style>
    """,
    unsafe_allow_html=True,
)


### FUNCTIONS###

def process_and_prepare_df(df):
    indonesia_holidays = holidays.Indonesia()
    df['Date'] = pd.to_datetime(df['Date'])

    # Add holiday feature
    df['is_holiday'] = df['Date'].apply(lambda x: x in indonesia_holidays or x.weekday() in [5, 6])

    # Add time-based features
    df['day_of_week'] = df['Date'].dt.weekday
    df['month'] = df['Date'].dt.month
    df['week_of_year'] = df['Date'].dt.isocalendar().week

    # Select relevant columns
    df_prepared = df[['Date', 'is_holiday', 'day_of_week', 'month', 'week_of_year', 'Occ (%) Sold']]

    return df_prepared

def get_viable_lag(df, nlags=196):
    series = df["Occ (%) Sold"]
    acf_values = sm.tsa.acf(series, nlags=nlags)
    conf_interval = 1.96 / np.sqrt(len(series))

    acf_lag = np.argmax((np.abs(acf_values[1:]) < conf_interval)) + 1

    return acf_lag

def forecast_occupancy(df):
    # Get lag value
    lags_A = int(get_viable_lag(df))

    df["Date"] = pd.to_datetime(df["Date"])
    last_date = df['Date'].max()

    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=28)
    future_extra = pd.DataFrame({'Date': future_dates, 'Occ (%) Sold': [None] * 28})

    future_df = pd.concat([df, future_extra], ignore_index=True)

    # Step 1: Process the data
    df_processed = process_and_prepare_df(future_df)

    df_processed_forecast = df_processed[-28:].copy()
    df_processed = df_processed[:-28]

    series_a = TimeSeries.from_dataframe(df_processed, time_col='Date', value_cols=['Occ (%) Sold', 'is_holiday', 'day_of_week', 'month', 'week_of_year'])
    series_a_forecast = TimeSeries.from_dataframe(df_processed_forecast, time_col='Date', value_cols=['Occ (%) Sold', 'is_holiday', 'day_of_week', 'month', 'week_of_year'])

    # Step 2: Train/Test Split
    train_size = int(len(series_a) * 0.8)
    train_a = series_a[:train_size]
    test_a = series_a[train_size:]

    target_train_a = train_a['Occ (%) Sold']
    future_cov_train_a = train_a[['is_holiday', 'day_of_week', 'month', 'week_of_year']]

    target_test_a = test_a['Occ (%) Sold']
    future_cov_test_a = test_a[['is_holiday', 'day_of_week', 'month', 'week_of_year']]

    ## Forecast Setup
    target_series_a = series_a['Occ (%) Sold']
    future_cov_series_a = series_a[['is_holiday', 'day_of_week', 'month', 'week_of_year']]

    target_forecast_a = series_a_forecast['Occ (%) Sold']
    future_cov_forecast_a = series_a_forecast[['is_holiday', 'day_of_week', 'month', 'week_of_year']]

    # Step 3: Optuna Optimization
    def objective_a(trial):
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 50, 450),
            'max_depth': trial.suggest_int('max_depth', 1, 30),
            'num_leaves': trial.suggest_int('num_leaves', 7, 2047),
        }

        model_a = LightGBMModel(
            lags=lags_A,
            lags_future_covariates=[0, 1, 2, 3, 4, 5, 6],
            output_chunk_length=28,
            verbose=-1,
            random_state=42,
            **params
        )

        model_a.fit(target_train_a, future_covariates=future_cov_train_a)

        backtest_predictions = model_a.historical_forecasts(
            series=target_train_a.concatenate(target_test_a),
            future_covariates=future_cov_test_a,
            start=len(target_test_a),
            forecast_horizon=28,
            stride=28,
            retrain=False,
            verbose=False
        )

        return smape(target_test_a, backtest_predictions)

    study_a = optuna.create_study(direction='minimize')
    study_a.optimize(objective_a, n_trials=3)

    best_params_a = study_a.best_params
    print("Best hyperparameters for model_a:", best_params_a)

    # Train Final Model
    model_a_best = LightGBMModel(
        lags=lags_A,
        lags_future_covariates=[0, 1, 2, 3, 4, 5, 6],
        output_chunk_length=28,
        verbose=-1,
        random_state=42,
        **best_params_a
    )

    model_a_best.fit(target_series_a, future_covariates=future_cov_series_a)

    # Generate Final Predictions
    pred_a_best = model_a_best.predict(len(target_forecast_a), future_covariates=future_cov_forecast_a)

    return pred_a_best

indonesia_holidays = holidays.Indonesia()

def new_plot_interactive(predictions, num_points):
    hover_text = []
    marker_colors = []

    for i, date in enumerate(predictions.time_index):
        value = predictions.values()[i][0]
        is_holiday = date in indonesia_holidays
        is_weekend = date.weekday() >= 5

        hover_value = min(max(value, 0), 100)

        if is_holiday:
            holiday_name = indonesia_holidays[date]
            hover_text.append(f'{date.strftime("%Y-%m-%d")}<br>{date.day_name()}<br>Holiday: {holiday_name}<br>Occupancy: {hover_value:.2f}%')
        else:
            hover_text.append(f'{date.strftime("%Y-%m-%d")}<br>{date.day_name()}<br>Occupancy: {hover_value:.2f}%')

        if is_holiday or is_weekend:
            marker_colors.append('gold')
        else:
            marker_colors.append('blue')

    fig = go.Figure()

    new_predictions = predictions.values().flatten()
    new_predictions = np.clip(new_predictions, 0, 100)

    fig.add_trace(go.Scatter(
        x=predictions.time_index,
        y=new_predictions,
        mode='lines',
        line=dict(color='#333333', shape='spline'),
        fill='tozeroy',
        fillcolor='rgba(169, 169, 169, 0.4)',
        hoverinfo='text',
        text=hover_text,
    ))

    fig.add_trace(go.Scatter(
        x=predictions.time_index,
        y=new_predictions,
        mode='markers',
        marker=dict(color=marker_colors, size=10),
        text=hover_text,
        hoverinfo='text',
        name='Prediction Points'
    ))

    fig.update_layout(
        title='Occupancy Rate Forecasting',
        xaxis_title='Date',
        yaxis_title='Occupancy Rate',
        yaxis=dict(range=[0, 100]),
        hovermode='closest',
        xaxis=dict(
            tickmode='linear',
            dtick='D1',
            tickformat='%Y-%m-%d',
            tickangle=-45
        ),
        template='seaborn',
        showlegend=False
    )

    st.plotly_chart(fig)



# Home Page
if st.session_state.menu == "Home":
    st.title("Welcome to PT XYZ Hotel Forecasting System")
    st.write("""
    ### About This Application
    This system provides forecasts for quick and reliable predictions for our valued customers.

    ### How Can We Help You?
    - Upload your data to forecast your hotel's occupancy rate.
    - Gain clean visualizations of the occupancy rate for the next 28 days to help you plan ahead strategically.

    ### How to Use
    1. Press the button below to select the forecasting menu.
    2. Upload your Dataset from the XYZ Hotel System.
    3. Wait for the forecast results to load.
    """)

    if st.button("Go to Forecasting"):
        set_page("Forecasting")

# Forecasting Page
elif st.session_state.menu == "Forecasting":
    st.title("Forecasting")
    st.write("Upload your dataset")

    # Step 1: Upload user dataset
    uploaded_file = st.file_uploader("Upload your dataset", type=["csv"])

    if uploaded_file is not None:
        # Step 2: Read and preview the dataset
        df = pd.read_csv(uploaded_file)
        st.write("Dataset preview:")
        st.dataframe(df)

        # Step 3: Use predict_occupancy function to get predictions
        st.write("Running the forecasting model...")
        predictions = forecast_occupancy(df)

        # Step 4: Visualize the results
        st.write("Forecasting Plot:")
        new_plot_interactive(predictions, 28)

# About Us Page
elif st.session_state.menu == "About Us":
    st.title("About Us")
    st.write("""
    ### Company Information
    **PT XYZ** as an IT Solution provider, has been forming partnership with its client since 1988. Serving more than 600 customers, across 30 industries, and 250 line of business in South East Asia, by applying ISO 9001 for more than 20 years. Not only by the number and location of the clients, PT XYZ also keeps agile with the latest features of the product technology and trend of the digitalization era.

    ### Vision & Mission
    **Vision**
    To be the leading force in digital transformation, pioneering cutting-edge ICT solutions that drive innovation, efficiency, and global connectivity.

    **Mission**
    - Empower businesses with intelligent and scalable technology solutions that enhance productivity and sustainability.
    - Foster a culture of continuous innovation, ensuring clients stay ahead in the digital era.
    - Deliver exceptional ICT services with a commitment to quality, security, and customer satisfaction.
    - Expand our influence globally while maintaining a strong foundation of excellence in Indonesia.

    ### Contact Information
    - **Email**   : info@xyz.co.id
    - **Phone**   : +62 11 - 222 3333
    - **Address** : Jl. XYZ No. 2, Jakarta, Indonesia

    ---
    ### Copyright
    © 2024 PT XYZ. All rights reserved.
    """)


Deployment

In [ ]:
from pyngrok import ngrok

# Token ngrok dari akun sendiri
ngrok.set_auth_token("2njmUSZRKuXEy8GSPANSix1cCzf_v5xQATTAzUkpeRWNwSiU")

# Ngrok Tunnel
public_url = ngrok.connect("8501", "http")
print(f"Streamlit app running at: {public_url}")

# Run the app
!streamlit run app.py &